In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder , StandardScaler
import matplotlib.pyplot as plt


# Importer le DataFrame depuis le fichier CSV
df = pd.read_csv('dataframe.csv')

In [3]:
# Étape 1 : Nettoyage initial
def nettoyage_initial(df):
    # Suppression des colonnes et lignes avec plus de 95% de valeurs manquantes
    seuil = 0.95
    df = df.loc[:, df.isna().mean() < seuil]
    seuil_lignes = 0.90
    df = df.dropna(axis=0, thresh=df.shape[1] * seuil_lignes)
    # Suppression de la colonne 'Patient ID'
    df = df.drop(columns=['Patient ID'])
    # Remplacer 'positive' par 1 et 'negative' par 0 dans la colonne 'SARS-Cov-2 exam result' pour travailler avec des valeurs numériques
    df['SARS-Cov-2 exam result'] = df['SARS-Cov-2 exam result'].replace({'positive': 1, 'negative': 0})
    return df

In [4]:
# Suppression des doublons 
def supprimer_doublons(df):
    df_sans_doublons = df.drop_duplicates()
    return df_sans_doublons

In [5]:
def supprimer_lignes_nan(df):
    df_sans_nan = df.dropna(axis=0)
    return df_sans_nan

In [6]:
def encodage_string(df):
    colonnes_string = df.select_dtypes(include=['object']).columns
    label_encoder = LabelEncoder()
    for col in colonnes_string:
        df[col] = label_encoder.fit_transform(df[col].astype(str))
    return df

In [7]:
def supprimer_valeurs_aberrantes(df, seuil=1.5):
    # Calculer les limites inférieure et supérieure pour chaque colonne
    limites_inferieures = df.quantile(0.25) - seuil * (df.quantile(0.75) - df.quantile(0.25))
    limites_superieures = df.quantile(0.75) + seuil * (df.quantile(0.75) - df.quantile(0.25))
    # Supprimer les lignes contenant des valeurs aberrantes
    masque = ((df < limites_inferieures) | (df > limites_superieures)).any(axis=1)
    df_clean = df[~masque]
    return df_clean


In [ ]:
from outils import visualiser_valeurs_aberrantes

df = nettoyage_initial(df)
df_sans_nan = supprimer_lignes_nan(df)
df_encode = encodage_string(df_sans_nan)

visualiser_valeurs_aberrantes(df_encode)
supprimer_valeurs_aberrantes(df_encode, seuil=1.5)
visualiser_valeurs_aberrantes(df_encode)

 # Vérifier les modifications
print(df['SARS-Cov-2 exam result'].head())

# Définir l'affichage pour afficher toutes les colonnes sur une seule ligne
pd.set_option("display.max_columns", None)

# Afficher le DataFrame avec toutes les colonnes sur une seule ligne
print(df.head().to_string(index=False))